# Scenario 1: Price Surge¶****

In [ ]:
# Define demand function
def demand_function(price):
    """Demand function for EV owners based on price."""
    return max(100 - 5 * price, 0)


In [ ]:
import pandas as pd
import numpy as np

def demand_function(price):
    """Define a demand function based on price."""
    base_demand = 100  # Base demand at zero price
    price_sensitivity = 10  # How sensitive demand is to price changes
    return max(0, base_demand - (price * price_sensitivity))

def price_surge_simulation(station_data):
    """Simulate a price surge at selected stations using a Stackelberg model."""
    surge_price = 2.0  # Increase price significantly for testing
    affected_stations = station_data.sample(frac=0.1)  # Randomly select 10% of stations

    # Create a copy of the original station data to track user behavior
    updated_station_data = station_data.copy()

    # Apply price surge to affected stations
    for index, station in affected_stations.iterrows():
        updated_station_data.at[index, 'dollars'] *= surge_price  # Apply price surge

    # Calculate new demand based on the updated prices
    updated_station_data['new_demand'] = updated_station_data['dollars'].apply(demand_function)

    # Simulate user switching behavior based on price thresholds
    for index, station in affected_stations.iterrows():
        # Simulate behavior for users at this station
        current_demand = updated_station_data.at[index, 'new_demand']
        alternative_stations = updated_station_data[updated_station_data['stationId'] != station['stationId']]

        # Assume a threshold where users will switch if the price is too high
        switch_threshold = 1.5 * station['dollars']  # 50% higher than the original price
        switched_users = alternative_stations[alternative_stations['dollars'] < switch_threshold]

        # Adjust demand for the station based on how many switched
        updated_station_data.at[index, 'new_demand'] -= len(switched_users)

    return updated_station_data

# Example usage:
station_data = pd.DataFrame({
    'stationId': range(1, 101),
    'dollars': np.random.uniform(0.5, 1.5, 100)  # Random prices between $0.5 and $1.5
})

# Run the price surge simulation
updated_station_data = price_surge_simulation(station_data)

# Display the results
print(updated_station_data[['stationId', 'dollars', 'new_demand']])


In [ ]:
import matplotlib.pyplot as plt

def plot_price_vs_demand(updated_station_data):
    """Plot the relationship between price and new demand."""
    plt.figure(figsize=(12, 6))
    
    # Scatter plot of prices vs new demand
    plt.scatter(updated_station_data['dollars'], updated_station_data['new_demand'], color='blue', alpha=0.7)

    # Adding a trend line
    z = np.polyfit(updated_station_data['dollars'], updated_station_data['new_demand'], 1)
    p = np.poly1d(z)
    plt.plot(updated_station_data['dollars'], p(updated_station_data['dollars']), color='red', linestyle='--', label='Trend Line')

    # Titles and labels
    plt.title('Price vs. New Demand for EV Charging Stations')
    plt.xlabel('Price ($)')
    plt.ylabel('New Demand')
    plt.xlim(0, updated_station_data['dollars'].max() + 0.5)
    plt.ylim(0, updated_station_data['new_demand'].max() + 10)
    plt.grid()
    plt.legend()
    plt.show()

# Call the plotting function
plot_price_vs_demand(updated_station_data)
